NLP project

In [1]:
import sys  
sys.path.insert(1, './transformer')

In [10]:
from dataset import *
from Transformer import *
from utils import *

In [3]:
input_path = './data/gec-only/train.src.tok'
target_path = './data/gec-only/train.tgt.tok'

dataset = construct_dataset(input_path, target_path, 856*2, 16)

In [4]:
len(dataset)

2047

In [5]:
for (batch, (input_ids, target_ids)) in enumerate(dataset):
    print(batch)
    print(input_ids)
    print(target_ids)
    break

0
tf.Tensor(
[[    0 10431 47489 ...     1     1     1]
 [    0 47447    13 ...     1     1     1]
 [    0 25417 17772 ...     1     1     1]
 ...
 [    0 36765 25482 ...     1     1     1]
 [    0 36709 34251 ...     1     1     1]
 [    0 25417 10965 ...     1     1     1]], shape=(16, 1711), dtype=int32)
tf.Tensor(
[[10431 47489 50118 ...     1     1     1]
 [47447    13  1470 ...     1     1     1]
 [25417 17772 36709 ...     1     1     1]
 ...
 [36765 25482 36709 ...     1     1     1]
 [36709 34251 22063 ...     1     1     1]
 [25417 10965 41171 ...     1     1     1]], shape=(16, 1711), dtype=int32)


In [11]:
# Initialize the Transformer model
num_layers = 6
d_model = 512
num_heads = 8
dff = 2048
target_vocab_size = len(dataset)
pe_target = 1711
rate = 0.01
target_vocab_size = len(target_ids)
transformer_model = Transformer(num_layers, d_model, num_heads, dff,
                                target_vocab_size, pe_target, rate)


loading file vocab.json from cache at C:\Users\a.hak/.cache\huggingface\hub\models--distilroberta-base\snapshots\d5411c3ee9e1793fd9ef58390b40a80a4c10df32\vocab.json
loading file merges.txt from cache at C:\Users\a.hak/.cache\huggingface\hub\models--distilroberta-base\snapshots\d5411c3ee9e1793fd9ef58390b40a80a4c10df32\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading file tokenizer.json from cache at C:\Users\a.hak/.cache\huggingface\hub\models--distilroberta-base\snapshots\d5411c3ee9e1793fd9ef58390b40a80a4c10df32\tokenizer.json
loading configuration file config.json from cache at C:\Users\a.hak/.cache\huggingface\hub\models--distilroberta-base\snapshots\d5411c3ee9e1793fd9ef58390b40a80a4c10df32\config.json
Model config RobertaConfig {
  "_name_or_path": "distilroberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [13]:
# Define loss function, optimizer, and other necessary components (as shown in the previous response)

# Training step function
@tf.function
def train_step(input_ids, target_ids):
    tar_inp = target_ids[:, :-1]
    tar_real = target_ids[:, 1:]

    # Create masks
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(input_ids, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer_model(input_ids, tar_inp, True, enc_padding_mask, combined_mask, dec_padding_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer_model.trainable_variables))

    return loss

In [14]:
# Training loop
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0

    for (batch, (input_ids, target_ids)) in enumerate(dataset):  # Assuming dataset is prepared
        batch_loss = train_step(input_ids, target_ids)
        total_loss += batch_loss

        if batch % 100 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {batch_loss.numpy():.4f}')

    print(f'Epoch {epoch + 1} Loss {total_loss / (batch + 1):.4f}')

OperatorNotAllowedInGraphError: in user code:

    File "C:\Users\a.hak\AppData\Local\Temp\ipykernel_55008\3181115009.py", line 13, in train_step  *
        predictions, _ = transformer_model(input_ids, tar_inp, True, enc_padding_mask, combined_mask, dec_padding_mask)
    File "C:\Users\a.hak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    OperatorNotAllowedInGraphError: Exception encountered when calling layer 'transformer' (type Transformer).
    
    in user code:
    
        File "C:\Users\a.hak\JupiterNotebooks\nlp\./transformer\Transformer.py", line 21, in call  *
            dec_output, attention_weights = self.decoder(
        File "C:\Users\a.hak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
            raise e.with_traceback(filtered_tb) from None
    
        OperatorNot